# Numerical methods for solving IVP's

IVP (initial value problem) is an ODE given with an initial condition, which is the value of the unknown function at a point in its domain. An IVP can be denoted as below:

$$u'(x)=f(x,u(x)) \\ u\left( x_{0} \right) = u_{0}$$

This notebook will only be about popular numerical methods for solving IVPs. There are different methods for solving BVPs (boundary value problems).

We can use numerical methods to approximate IVP's with ODEs that can't be solved using analytical methods.

## Lipschitz condition

It is important to know if the given IVP has a solution and whether if this solution is unique. Given $R={(x, u): x_0 \le x \le b, c \le u \le d} \subseteq \mathbb{R}$, suppose that $f(x,u)$ is continuous on $R$. If $f(x,u)$ satisfies the Lipschitz condition on $R$ in the variable $u$, then the IVP has a unique solution $u(x)$ for  $x_0 \le x \le b$ and $c \le u \le d$.

$f(x,u)$ satisfies a Lipschitz condition in the variable $u$ on $\mathbb{R}$ if a constant $L \gt 0$ exists such that $|f(x,u_1)-f(x,u_2)| \le L|u_1-u_2|$ for $(x,u_1),(x,u_2) \in R$. $L$ is called the Lipschitz constant.

An alternative characterization of the Lipschitz constant is, suppose $f(x,u)$ is defined on $R$. If there is a constant $L \gt 0$ such that 

$$
\left|\frac{df(x,u)}{du}\right| \le L  \text{, for all } (x,u) \in \mathbb{R}$$

then Lipschitz condition is satisfied with the Lipschitz constant $L$.

## Euler's method
Euler's method approximates values of unknown function $u$ using tangent lines. Firstly, the x axis is divided into $n$ intervals of length of $h$ such that $x_{n}=x_{0} + nh=x_{n-1}+h$. Then, for each beginning point $x_{n}$ of these intervals, a line $v$ that is tangent to the curve of the unknown $u=u(x)$ is considered. $v_{n+1}$, the value of $v$ at the terminal point of the interval, $x_{n+1}$ is the approximation of $u$ at $x_{n+1}$. The approximation can be formulated as below:
$$u(x_{n+1}) \approx v_{n+1}=v_{n}+hf(x_n, v(x_n))\\ v_0=u(x_0)=u_0$$

The last term in the above formula, $hf(x, u(x))$ is the vertical length of the small triangle created by the tangent line between $x_{n}$ and $x_{n+1}$. If we call this height $H$, $H/h=tan(\theta)=u'=f(x,u(x))$, thus when both sides are multiplied by $h$, the last term is obtained.

Larger values of $n$ and smaller values of $h$ will lead to more accurate approximations.

In [1]:
def EulerMethod(x0, u0, h, n, f):
    x = [x0 + _n*h for _n in range(0, n+2)]
    v = [0 for i in range(0, n+2)]
    v[0] = u0
    
    for _n in range(1, n+2):
        v[_n] = v[_n-1] + h * f(x[_n-1], v[_n-1])
        
    return v

**Example:** Given $u'=x-u$ and $(x_0, u_0)=(0, 1)$. Approximate values of u at $0.1$, $0.2$ and $0.3$ using Euler's method. Here, $f(x,u(x))=x-u$ and $h=0.1$.

In [2]:
f = lambda x, u: x - u

#There are 2 intervals, (0.1, 0.2) and (0.2 , 0.3)
approximations = EulerMethod(0, 1, 0.1, 2, f)
print(approximations)

[1, 0.9, 0.8200000000000001, 0.758]


## Heun's method 

Heun's method is an attempt to reduce error of Euler's method without changing $n$ and $h$. Using the fundamental theorem of calculus, we know that:

$$\int_{x_n}^{x_{n+1}}f(x,u(x))dx = \int_{x_n}^{x_{n+1}}\frac{du}{dx}dx = u(x_{n+1})-u(x_n)$$

We don't know the value of $u(x_{n+1})$, so we use an approximation for it using the Euler's method, and use trapezoid rule for approximating the integral on the leftmost side of the equality above. Thus we formulize the Heun's method as:

$$u(x_{n+1})\approx v_{n+1} = v_{n} + \frac{h}{2} \left[f(x_n,v(x_n)) + f(x_{n+1},v(x_n)+hf(x_n, v(x_n))\right]\\ v_0=u(x_0)=u_0$$

In [3]:
def HeunMethod(x0, u0, h, n, f):
    x = [x0 + _n*h for _n in range(0, n+2)]
    v = [0 for i in range(0, n+2)]
    v[0] = u0

    for _n in range(1, n+2):
        v[_n] = v[_n-1] + (h/2) * (f(x[_n-1], v[_n-1]) + f(x[_n], v[_n-1]+ h * f(x[_n-1], v[_n-1])))
        
    return v

**Example:** Given the same IVP above, let's approximate the same points again using Heun's method.

In [4]:
approximations = HeunMethod(0, 1, 0.1, 2, f)
print(approximations)

[1, 0.91, 0.8380500000000001, 0.78243525]


## Picard's method
Deriving a bounded function may result in an unbounded function whose value exceeds a number at any point. Derivative of functions like this may have fastly increasing or decreasing values. This makes finite difference approximations for these functions problematic. Same characteristic is observed with solutions of differential equations, when they become unbounded at some point. Picard's method handles this by expressing the IVP as an integral equation.

Picard's method is an iterative/successive method for solving IVPs. This means with each succession the approximation becomes more accurate and coverges to the actual result.

An IVP can be written in an integral equation form: 

$$u(x) \approx \phi_{n+1}(x)=u(x_0)+\int_{x_0}^{x}f(t,\phi_{n}(t))dt\\ \lim_{n \to \infty}\phi_n(x)=u(x) \\ \phi_0=u(x_0)=u_0$$

In [5]:
from sympy import *

def PicardMethod(x0, u0, f, _x, n_iterations):
    x, t = symbols('x t')
    approximation = u0 + 0*x
    
    for i in range(n_iterations):
        integrand = f.subs([(x, t),(u,approximation.subs(x, t))])
        approximation = u0 + integrate(integrand, (t, x0, x))
    
    return approximation.subs(x, _x)

**Example:** Given the same IVP above, let's approximate same points again using Picard's method for 50 iterations.

In [6]:
x,u = symbols('x, u')
f = x - u

print(PicardMethod(0, 1, f, 0.1, 50))
print(PicardMethod(0, 1, f, 0.2, 50))
print(PicardMethod(0, 1, f, 0.3, 50))

0.909674836071919
0.837461506155964
0.781636441363436


## Taylor's method of higher orders

This method approximates the solution/unknown function $u(x)$ by expressing it as $m^{th}$ order Taylor polynomial.

$$
u(x_{n+1}) = u(x_n) + \left[\sum_{i=1}^{m} \frac{u^{(i)}(x_n)(x_{n+1}-x_n)^{m+1}}{i!}\right] + \frac{u^{(m+1)}(\xi)(x_{n+1}-x_n)^{m+1}}{(m+1)!}
$$

Knowing that $u'=f(x,u)$, we know that $u''=f'(x,u), u''=f'''(x,u), \ldots, u^{(n)}=f^{(n-1)}(x,u)$. If we substitute $u$'s with respective $f$'s and omit the truncation term (which is the rightmost term on the expansion above), we obtain:

$$u(x_{n+1}) \approx v(x_{n+1}) = v(x_n) + \left[\sum_{i=1}^{m} \frac{f^{(i-1)}(x_n, v(x_n))(x_{n+1}-x_n)^i}{i!}\right] \\
v(x_0) = u(x_0) = u_0 \\
x_{n+1}-x_n=h$$

In [7]:
def TaylorMethod(x0, u0, h, n, f, order):
    x, u = symbols('x u')
    _x = [x0 + _n*h for _n in range(n+2)]
    _v = [0 for _n in range(n+2)]
    _v[0] = u0
    
    for _n in range(1, n+2):
        taylor = 0
        
        for i in range(1, order+1):
            deriv = f.diff(x, i-1, u, i-1).subs([(x,_x[_n-1]),(u,_v[_n-1])])
            taylor += (deriv * h**i)/factorial(i)
            
        _v[_n] = _v[_n-1] + taylor
    
    return _v

**Example:** Given the same IVP above, let's approximate the same points again using Taylor's method of 3rd order.

In [8]:
print(TaylorMethod(0, 1, 0.1, 2, f, 3))

[1, 0.900000000000000, 0.820000000000000, 0.758000000000000]


## Runge-Kutta method of 4th order

Taylor's method requires computation of derivatives of $f$. The Runge-Kutta methods avoid this difficulty by mimicking Taylor's method. Let us denote the approximation formula of Taylor's method of order m $T_m(x_n)$. This method can be formulated as:

$$u_{n+1} \approx T_m(x_n) \approx v(x_{n+1}) = v(x_n) + (w_1K_1+w_2K_2+ \ldots +w_mK_m) \\ v(x_0) = u(x_0) = u_0$$

where $w_1, w_2, \ldots, w_n$ are parameters which must be determined such that $\sum_nw_n=1$. $K_1, K_2, \ldots, K_n$ are recursive evaluations of approximations of the slope of $u(x)$. 

Runge-Kutta method of order 4 (RK4) is the most commonly used Runge-Kutta method. Like Taylor's method, we start off by using fundamental theorem of calculus but approximating the integral of $f(x,u)$ using Simpson's method this time. 
$$u(x_{n+1})-u(x_n) \approx \frac{h}{6} \left[f(x_n, u(x_n)) + 4f(x_n + \frac{h}{2}, u(x_n + \frac{h}{2})) + f(x_{n+1}, u(x_{n+1}))\right] (\star)$$ 

Then, we divide our $x$ axis into two intervals, $(x_n, x_n+\frac{h}{2})$ and $(x_n+\frac{h}{2}, x_{n+1})$. Then we recursively approximate slope values at remaining points on the intervals. $K_1$ is the actual slope at point $(x_n, u_n)$ since we know $u'=f(x,u)$. $K_2$ and $K_3$ are used to approximate slope at the midpoint. $K_4$ is the approximation of the slope at terminal point $(x_{n+1}, u_{n+1})$. 

$$K_1 = f(x_n, u(x_n))\\
K_2 = f(x_n + \frac{h}{2}, u_n + \frac{h}{2}K_1)\\
K_3 = f(x_n + \frac{h}{2}, u_n + \frac{h}{2}K_2)\\
K_4 = f(x_{n}+h, u_n + hK_3))\\ 
f(x_n+\frac{h}{2}, u(x_n+\frac{h}{2})) = \frac{K_2+K_3}{2}$$ 

Substituting these values on the equation $(\star)$ , we obtain:
$$u(x_{n+1}) \approx v(x_{n+1}) = v(x_n) + \frac{h}{6} \left[K_1+2K_2+2K_3+K_4 \right]
\\ v(x_0) = u(x_0) = u_0
$$



In [9]:
def RK4(x0, u0, h, n, f):
    x = [x0 + _n*h for _n in range(0, n+2)]
    v = [0 for i in range(0, n+2)]
    v[0] = u0
    
    for _n in range(1, n+2):
        K1 = f((x[_n-1]), (v[_n-1]))
        K2 = f((x[_n-1]+h/2), (v[_n-1]+K1*h/2))
        K3 = f((x[_n-1]+h/2), (v[_n-1]+K2*h/2))
        K4 = f((x[_n-1]+h), (v[_n-1]+h*K3))
        
        v[_n] = v[_n-1] + (h/6)*(K1+2*K2+2*K3+K4)
        
    return v

**Example:** Given the same IVP above, let's approximate the same points again using Runge-Kutta method of 4th order:

In [10]:
f = lambda x, u: x - u
print(RK4(0, 1, 0.1, 2, f))

[1, 0.909675, 0.8374618028125, 0.7816368440023556]


## Exact solution

Exact solution to the example IVP is:
$$u(x)=x+2e^{-x}-1$$

The exact values at points $0.1$, $0.2$ and $0.3$ are:

In [11]:
exact = lambda x: x + 2 * exp(-x) - 1 

for x in range(1, 4):
    x *= 0.1
    print(f"x: {x} u(x): {exact(x)}")

x: 0.1 u(x): 0.909674836071919
x: 0.2 u(x): 0.837461506155964
x: 0.30000000000000004 u(x): 0.781636441363436


## Result comparison

| Method | x=0.1 | x=0.2 | x=0.3 | 
| --- | --- | --- | --- |
| Exact solution | 0.909674836071919 | 0.837461506155964 | 0.781636441363436 |
| Euler's method | 0.9 | 0.8200000000000001 | 0.758 |
| Heun's method | 0.91 | 0.8380500000000001 | 0.78243525 |
| Picard's method (n=50) | 0.909674836071919 | 0.837461506155964 | 0.781636441363436 |
| Taylor's method (3rd order) | 0.900000000000000 | 0.820000000000000 | 0.758000000000000 |
| RK4 | 0.909675 | 0.8374618028125 | 0.7816368440023556 |